<a href="https://colab.research.google.com/github/srzsth-create/DA-course/blob/main/Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#import the data
df =pd.read_csv("synthetic_retail_sales_2021_2023.csv")

In [ ]:
#show the data
df

,date,store_id,region,product,category,units_sold,unit_price,revenue,promotion_flag
0,2022-11-13,EA-S04,East,Beta Widget,Widgets,3,68.47,205.41,0
1,2021-11-11,NO-S02,North,Alpha Widget,Widgets,2,73.72,147.44,0
2,2022-12-06,WE-S03,West,Epsilon Gadget,Gadgets,2,123.66,247.32,0
3,2022-08-07,WE-S02,West,Epsilon Gadget,Gadgets,3,86.19,258.57,1
4,2022-05-21,CE-S05,Central,Gamma Widget,Widgets,4,47.80,191.20,0
...,...,...,...,...,...,...,...,...,...
65681,2022-09-12,NO-S04,North,Rho Part,Parts,3,12.09,36.27,0
65682,2021-04-15,EA-S02,East,Xi Module,Modules,3,222.17,666.51,0
65683,2023-07-04,NO-S06,North,Beta Widget,Widgets,6,49.89,299.34,0
65684,2021-01-15,SO-S02,South,Upsilon Bundle,Bundles,1,473.69,473.69,0


In [ ]:
df.columns

Index(['date', 'store_id', 'region', 'product', 'category', 'units_sold',
       'unit_price', 'revenue', 'promotion_flag'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65686 entries, 0 to 65685
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            65686 non-null  object 
 1   store_id        65686 non-null  object 
 2   region          65686 non-null  object 
 3   product         65686 non-null  object 
 4   category        65686 non-null  object 
 5   units_sold      65686 non-null  int64  
 6   unit_price      65686 non-null  float64
 7   revenue         65044 non-null  float64
 8   promotion_flag  65686 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 4.5+ MB


In [ ]:
df.shape # It provides number of rows and columns

(65686, 9)

In [ ]:
df.describe() # It provides statistical summary of the data

,units_sold,unit_price,revenue,promotion_flag
count,65686.000000,65686.00000,65044.000000,65686.000000
mean,3.016503,109.24617,329.270056,0.051609
std,1.710448,114.44540,438.000867,0.221238
min,-9.000000,6.70000,-2602.980000,0.000000
25%,2.000000,26.12000,63.497500,0.000000
50%,3.000000,57.22000,162.375000,0.000000
75%,4.000000,166.84250,412.117500,0.000000
max,12.000000,719.55000,6591.100000,1.000000


In [ ]:
df.isna().sum()
#check for missing or null values


,0
date,0
store_id,0
region,0
product,0
category,0
units_sold,0
unit_price,0
revenue,642
promotion_flag,0


In [ ]:
print("Missing Values:")
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])


Missing Values:
revenue    642
dtype: int64


In [ ]:
# Create a copy for transformation
df_clean = df.copy()

# Check for missing revenue and fill with calculated values
missing_revenue = df_clean['revenue'].isnull().sum()
print(f"Missing revenue records before cleaning: {missing_revenue}")

if missing_revenue > 0:
    df_clean['revenue'] = df_clean['revenue'].fillna(
        df_clean['units_sold'] * df_clean['unit_price']
    )
    print(f"Missing revenue records after cleaning: {df_clean['revenue'].isnull().sum()}")


Missing revenue records before cleaning: 642
Missing revenue records after cleaning: 0


In [ ]:
#outliers -- for detection of that we will use the Interquartile Range (IQR) method

def detect_outliers_iqr(df, revenue):
    Q1 = df[revenue].quantile(0.25)
    Q3 = df[revenue].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[revenue] < lower_bound) | (df[revenue] > upper_bound)]
    return outliers, Q1, Q3, lower_bound, upper_bound

outliers, Q1, Q3, lower_bound, upper_bound = detect_outliers_iqr(df, 'revenue')

print("Q1:", Q1)
print("Q3:", Q3)
print("Lower Bound:", lower_bound)
print("Upper Bound:", upper_bound)

print("Outliers detected using IQR method:", outliers)
print("Number of outliers detected:", len(outliers))

# print(detect_outliers_iqr(df, 'revenue')


Q1: 63.4975
Q3: 412.1175
Lower Bound: -459.43250000000006
Upper Bound: 935.0475000000001
Outliers detected using IQR method:              date store_id   region         product category  units_sold  \
14     2022-08-24   EA-S03     East         Tau Kit     Kits           4   
22     2023-03-09   SO-S06    South         Tau Kit     Kits           6   
23     2021-07-17   CE-S01  Central       Sigma Kit     Kits          10   
24     2023-12-01   CE-S06  Central       Sigma Kit     Kits           7   
44     2023-03-14   CE-S04  Central  Upsilon Bundle  Bundles           3   
...           ...      ...      ...             ...      ...         ...   
65621  2022-07-06   NO-S04    North       Sigma Kit     Kits           4   
65641  2021-01-17   CE-S01  Central  Upsilon Bundle  Bundles           5   
65656  2022-12-18   WE-S06     West         Tau Kit     Kits           5   
65671  2021-03-30   WE-S05     West       Xi Module  Modules           6   
65677  2022-11-16   NO-S02    North  Up